In [31]:
!pip install -q nltk rouge-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# 루브릭

|평가문항|	상세기준|
|:---|:---|
|1. 기존 데이터셋을 추가 정제하고, generation 성능을 끌어올리기 위한 기법들을 실험해 모델 perfomance를 향상시켜보았는가?|	기존 데이터셋의 문제점을 분석하고 전처리 전략을 수립해 추가 정제를 진행했다. Beam search, Top-k(p) sampling 등 최선의 디코딩 전략을 수립해 향상된 모델 추론 결과를 제시했다. BLEU, ROUGE 등 생성된 텍스트를 평가하기 위한 메트릭을 적용한 정량적인 평가 결과와 주관적인 평가를 비교분석하였다.|
|2. 새로운 데이터를 수집해 전처리를 수행하여 모델을 재학습시켜보았는가?	|모두의 말뭉치, AI hub 등에 공개된 데이터를 사용해 추가 데이터셋을 구축하기 위한 기준과 근거를 수립했다. ChatGPT API나 다양한 한국어 benchmark 데이터셋을 활용해 Human Feedback 을 대체할 수 있는 아이디어를 구현했다. 위를 바탕으로 SFT, RM, PPO 세 단계에 필요한 각 데이터셋을 적절히 구축하여, 모델 추론 결과와 수립한 가설을 비교해보았다.|
|3. 학습 전략 또는 foundation model을 변경해 모델을 재학습시켜보았는가?	|더 적절한 Instruction Tuning 기법을 적용해 SFT를 해보거나, Reward Model의 ranking algorithm을 개선해보았다. KoGPT-2가 아닌 다른 모델을 initial model로 사용하여 모델 학습을 성공시켰다. 허깅페이스의 accelerate, bitsandbytes 라이브러리 등을 사용하여 더 큰 스케일의 모델로 ChatGPT를 re-building해 모델 성능을 향상시켰다.|

# 개선 방향 제시

1. 우리가 지난시간 살펴본 KoChatGPT 모델에 사용한 데이터셋은 아직 완벽히 정제되지 않았습니다.

2. Hunman Feedback이 반영된 데이터셋을 대체하기 위해
SFT와 RM 모델에 사용할 다양한 benchmark 데이터셋도 검토해볼 수 있습니다.

3. 언어모델의 생성능력을 좌우하는 최선의 디코딩을 위한 하이퍼파라미터 서치가 필요합니다.

4. 생성된 답변에 대한 주관적인 평가를 보완할 수 있는 정량적인 메트릭은 도입하지 않았었습니다.

5. LLM Trend Note1에서 살펴본 다양한 Instruction Tuning 및 Prompting 기법들도 적용해볼만 합니다.

6. 무엇보다 foundation model로 사용한 KoGPT-2는 Emergent abilities를 기대하기엔 다소 작은 사이즈의 모델입니다.
더 큰 파라미터 스케일을 가진 모델을 사용해보거나,

7. 더 효율적인 연산을 수행할 수 있는 LoRA의 적용 또는
새로운 Instruction Tuning 및 reward ranking 알고리즘을 도입해볼 수도 있습니다.

# 00. 임포트

In [32]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

# 00. 모델 & 토크나이저 불러오기

In [33]:
model_name = 'skt/kogpt2-base-v2' # skt/ko-gpt-trinity-1.2B-v0.5

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    bos_token='</s>', 
    eos_token='</s>', 
    unk_token='</s>', 
    pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


# 00. 데이터 & 데이터셋

In [34]:
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        data_path_1_SFT = 'data_kochatgpt/kochatgpt_1_SFT.jsonl'
        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [35]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [36]:
train_dataset = SFT_dataset(data_path_1_SFT='kochatgpt_1_SFT.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

# 00. Train

In [37]:
training_args = TrainingArguments(
    output_dir = "aiffel/KoChatGPT/test",    # 학습 결과를 저장할 디렉토리 경로 지정 
    overwrite_output_dir = True,     # 기존 디렉토리가 있는 경우 덮어쓸지 여부 설정 (True: 덮어쓰기, False: 덮어쓰지 않음) 
    num_train_epochs = 3,    # 전체 학습 에포크 수 설정  
    per_device_train_batch_size = 8,    # 각 디바이스당 학습 배치 크기 설정 (GPU 또는 TPU 기준)
    per_device_eval_batch_size = 8,    # 각 디바이스당 평가 배치 크기 설정 (GPU 또는 TPU 기준)    
    warmup_steps = 5,    # 학습 시작 시 learning rate를 증가시키는 스텝 수 설정 (웜업 스텝)
    prediction_loss_only = True,    # 예측 손실만 계산할 것인지 설정 (True: 예측 손실만 계산, False: 전체 손실 계산)
    fp16 = True,    # Mixed precision 학습 사용 여부 설정 (True: 사용, False: 사용하지 않음)
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [38]:
trainer.train()
model.save_pretrained('aiffel/KoChatGPT/output_1_SFT')

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.007800
1000,2.850000
1500,2.795700
2000,2.251900
2500,2.262400
3000,2.240000
3500,1.827800
4000,1.840500
4500,1.823100


# 00. 생성

In [41]:
generator = pipeline('text-generation', model='aiffel/KoChatGPT/output_1_SFT', tokenizer=tokenizer)

generation_args = dict(   
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n   
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]
list_result = generator(list_prompt, **generation_args)   

/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [76]:
list_data_label = [
    '저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.',
    '1953년입니다.',
    '시카고 오헤어 국제공항은 미국 일리노이 주 시카고에 위치해 있습니다.',
    '미세먼지 농도는 어제와 비교해서 개선되었지만 아직도 나쁜 수준이며, 마스크 착용과 실외 활동 자제를 권장합니다. 정확한 미세먼지 농도를 확인하려면 해당 지역의 미세먼지 측정소에서 확인해보시기 바랍니다.',
]

In [101]:
for prompt, result, label in zip(list_prompt, list_result, list_data_label):
    q = prompt[prompt.index('\n')+1:prompt.rindex('\n')]
    print('Q.', q )
    g = result[0]['generated_text'][result[0]['generated_text'].rindex('(응답):')+5:]
    print('G.', g )
    print()
    a = label # 정답
    print('A.', label)
    print()
    print("1-Gram BLEU :", sentence_bleu(a.split(), g.split(), weights=(1, 0, 0, 0 )))  
    print("2-Gram BLEU :", sentence_bleu(a.split(), g.split(), weights=(0, 1, 0 ,0 )))  
    print("3-Gram BLEU :", sentence_bleu(a.split(), g.split(), weights=(0, 0, 1 ,0 )))  
    print("4-Gram BLEU :", sentence_bleu(a.split(), g.split(), weights=(0, 0, 0, 1 ))) 
    print('-'*50)

Q. 불고기용 고기 한우에요?

G. '저는 AI 어시스턴트이기 때문에 실제로 고기를 먹지 않습니다. 하지만 일반적으로 불고기용 고기는 한국인들이 즐겨 먹는 음식 중 하나입니다.實)實)은 중국어로 "불고기"를 의미합니다.實은 고기를 뜻하는 단어이며, 실제 고기를 먹지 않는다는 뜻입니다.實은 일본어로 "

A. 저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.

1-Gram BLEU : 0
2-Gram BLEU : 0
3-Gram BLEU : 0
4-Gram BLEU : 0
--------------------------------------------------
Q. 리처드 닉슨이 43대 부통령직을 수행한 년도는?

G. '리처드 닉슨은 41대 부통령직을 수행했습니다.辰巳, please provide more context or information about the statement. 따르면, 39대 부통령직은 리처드 닉슨이 47대 부통령을 수행한 년도의 연도를 의미합니다.辰

A. 1953년입니다.

1-Gram BLEU : 0
2-Gram BLEU : 0
3-Gram BLEU : 0
4-Gram BLEU : 0
--------------------------------------------------
Q. 시카고 오헤어 국제공항은 어디에 있어?

G. '시카고 오 헤어 국제공항은 미국 일리노이주 시카고에 위치해 있습니다.辰海道報道報道日報管理官報官報官保関係, please. "시카고 오헤이어 국제공항"이 무엇인지에 대한 정보가 제공되지 않아 정확한 답변을 제공할 수 없습니다.

A. 시카고 오헤어 국제공항은 미국 일리노이 주 시카고에 위치해 있습니다.

In [34]:
torch.cuda.empty_cache()

In [35]:
import gc
gc.collect()

26262